In [ ]:
import tensorflow as tf
import cv2
import numpy as np
from src.predict import preprocess, predict

MODEL = tf.keras.models.load_model("results/checkpoint_ft.keras")

In [ ]:
# 1) Carga y preprocess
img = cv2.imread("data/processed/val/bean_rust/bean_rust_1.jpg")
img = cv2.cvtColor(img, cv2.COLOR_BGR2RGB)
img = cv2.resize(img, (224,224))
x = np.expand_dims(img/255.0,0)

In [ ]:
# 2) Inferencia
preds = MODEL.predict(x)
idx = preds.argmax()
conf = preds[0,idx]
label = MODEL.class_names[idx] if hasattr(MODEL,"class_names") else idx
print(f"Predicción: {label} (conf: {conf:.2f})")

In [ ]:
# 3) Simulación de API REST con Flask
from flask import Flask, request, jsonify
app = Flask(__name__)

@app.route("/predict", methods=["POST"])
def predict_api():
    file = request.files["image"]
    data = file.read()
    # reutiliza tu función preprocess de src/predict.py
    x,_ = preprocess(data)
    p = MODEL.predict(x)[0]
    return jsonify({"label": int(p.argmax()), "confidence": float(p.max())})

In [ ]:
# Ejecuta:
# app.run(host="0.0.0.0", port=5000)